#  My project in altair and vega


In [190]:
import pandas as pd
import numpy as np
import altair as alt

In [191]:
license_url = "https://github.com/UIUC-iSchool-DataViz/is445_data/raw/main/licenses_fall2022.csv"
license = pd.read_csv(license_url)

In [192]:
license.head()

,_id,License Type,Description,License Number,License Status,Business,Title,First Name,Middle,Last Name,...,Specialty/Qualifier,Controlled Substance Schedule,Delegated Controlled Substance Schedule,Ever Disciplined,LastModifiedDate,Case Number,Action,Discipline Start Date,Discipline End Date,Discipline Reason
0,1189509,DETECTIVE BOARD,PERMANENT EMPLOYEE REGISTRATION,129446286,NOT RENEWED,N,NaN,EILEEN,NaN,SANTACRUZ,...,NaN,NaN,NaN,N,03/18/2022,NaN,NaN,NaN,NaN,NaN
1,801037,DETECTIVE BOARD,FIREARM CONTROL CARD,229030294.0,NOT RENEWED,N,NaN,DAGMAR,J,NORDLUND,...,NaN,NaN,NaN,N,08/16/2006,NaN,NaN,NaN,NaN,NaN
2,365129,COSMO,LICENSED COSMETOLOGIST,11053076.0,NOT RENEWED,N,NaN,RADOJE,NaN,ZELENOVIC,...,NaN,NaN,NaN,N,05/26/2006,NaN,NaN,NaN,NaN,NaN
3,595427,COSMO,LICENSED COSMETOLOGIST,11295645.0,ACTIVE,N,NaN,BECKY SUE,L,BURROUGHS,...,NaN,NaN,NaN,N,11/12/2021,NaN,NaN,NaN,NaN,NaN
4,653668,COSMO,LICENSED NAIL TECHNICIAN,169006247,NOT RENEWED,N,NaN,BILL G,L,LETNER,...,NaN,NaN,NaN,N,05/30/2006,NaN,NaN,NaN,NaN,NaN


In [193]:
license = license[["_id", "License Type", "Description", "License Status","Ever Disciplined", "Discipline Reason"]]

In [194]:
license.head()

,_id,License Type,Description,License Status,Ever Disciplined,Discipline Reason
0,1189509,DETECTIVE BOARD,PERMANENT EMPLOYEE REGISTRATION,NOT RENEWED,N,NaN
1,801037,DETECTIVE BOARD,FIREARM CONTROL CARD,NOT RENEWED,N,NaN
2,365129,COSMO,LICENSED COSMETOLOGIST,NOT RENEWED,N,NaN
3,595427,COSMO,LICENSED COSMETOLOGIST,ACTIVE,N,NaN
4,653668,COSMO,LICENSED NAIL TECHNICIAN,NOT RENEWED,N,NaN


In [195]:
license["License Status"].unique()

array(['NOT RENEWED', 'ACTIVE', 'CANCELLED', 'TERMINATED CARD RETURNED',
       'DECEASED', 'INACTIVE', 'EXPIRED', 'CLOSED',
       'TERMINATED WITHOUT CARD', 'INOPERATIVE',
       'TERMINATED VALID REASON', 'CHANGE OF OWNERSHIP',
       'Non Sufficient Fund Check Terminated'], dtype=object)

In [196]:
license['DisciplinedNum'] = license['Ever Disciplined'].replace({'Y': 1, 'N': 0}).astype('int8')

/var/folders/s0/sb498m3j5pq3p85ywy45wdpm0000gn/T/ipykernel_51964/3210570780.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  license['DisciplinedNum'] = license['Ever Disciplined'].replace({'Y': 1, 'N': 0}).astype('int8')


In [197]:
license_agg = (
    license.groupby("License Type", as_index=False)
           .agg(
               Total=("DisciplinedNum", "size"),
               Disciplined=("DisciplinedNum", "sum")
           )
)
license_agg_filtered = license_agg[license_agg["Disciplined"] > 0]


In [198]:
selector = alt.selection_single(fields=["License Type"])

bars = (
    alt.Chart(license_agg_filtered)
    .mark_rect()
    .encode(
        x=alt.X("License Type:N", sort="-y", axis=alt.Axis(labelAngle=45)),
        y=alt.Y("Disciplined:Q", title="Total Disciplined"),
        tooltip=["License Type", "Disciplined", "Total"]
    ).properties(width=400, title="Disciplinary Amounts").add_params(selector)
    
)

bars

/opt/anaconda3/lib/python3.12/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection_single' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [199]:
has_a_reason = license[license['Discipline Reason'].notna()]['Discipline Reason']

In [200]:
tax_keywords = ['tax', 'taxes']
crime_keywords = ['criminal', 'conviction', 'fraudulent', 'falsified','falsifying','substance','arrest']
qualification_keywords = ['qualification', 'without a license','unlicensed','expired','non-renewed']
billing_keywords = ['billing','insurance','child support','loan','non-bankable']
unprofessional_keywords = ['unprofessional','misconduct','probation']
def classify_reason(reason):
    r = reason.lower()

    if any(k in r for k in tax_keywords):
        return 'Taxes'
    if any(k in r for k in unprofessional_keywords):
        return 'Unprofessional'
    if any(k in r for k in billing_keywords):
        return 'Billing'
    if any(k in r for k in qualification_keywords):
        return 'Qualification'

    if any(k in r for k in crime_keywords):
        return 'Criminal/Conviction/Fraud'

    return 'other'
license['Reason Group'] = (
    has_a_reason
    .apply(classify_reason)
)

In [201]:
license['Reason Group'].value_counts()

Reason Group
Criminal/Conviction/Fraud    105
Taxes                         52
other                         43
Qualification                 33
Unprofessional                28
Billing                       26
Name: count, dtype: int64

In [202]:
details_agg = (
    license[license["Discipline Reason"].notna()]
    .groupby(["License Type", "Reason Group"], as_index=False)
    .agg(
        Count=("Discipline Reason", "size")
    )
)
details_agg.head(6)

,License Type,Reason Group,Count
0,ATHLETIC TRAINER,other,1
1,AUCTIONEER,other,1
2,COLLECTION AGENCY,Qualification,1
3,COLLECTION AGENCY,other,2
4,COMM ASSOC MGR,Taxes,1
5,COMM ASSOC MGR,other,1


In [203]:
details = (
    alt.Chart(details_agg)
    .mark_bar()
    .encode(
        x=alt.X("Reason Group:N", sort="-y",axis=alt.Axis(labelAngle=45)),
        y=alt.Y("Count:Q"),
        tooltip=["Reason Group", "Count"]
    )
    .properties(
        width=500, height=300,title="Reasons"
    ).transform_filter(
        selector
    )
)
chart = (bars) | (details)
chart

/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.HConcatChart(...)

In [204]:
myJekllDir = '/Users/jonahwitte/Desktop/IS_445/jonahlw2.github.io/assets/json/'

In [205]:
chart.save(myJekllDir + "altair_licenses.json")

/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
